<a href="https://colab.research.google.com/github/michel-wue/Seminar/blob/Tries_API_Data_extraction/plays_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests as r
import pandas as pd
from pandas import json_normalize
import re
from google.colab import drive
import datetime
from os import times

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
Data_basic = pd.DataFrame()
Data_basic = pd.DataFrame(columns=[
    'GameID','season','types',
    'week','HomeTeam_ID','AwayTeam_ID',
    'Winner'
    ])
week = list(range(1, 19))
seasons=list(range(2015,2023))

In [33]:


def infos(season,types,weeks):
  url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{season}/types/{types}/weeks/{weeks}/events"
  response = r.get(url)
  json_data = response.json()
  df = json_normalize(json_data['items'])
  for game in range(len(df)):
    rows=[]
    response = r.get(df['$ref'][game])
    data=response.json()
    try:
      df2=json_normalize(data['competitions'])
      rows.append(df2['id'][0])
      rows.append(season)
      rows.append(types)
      rows.append(weeks)
      rows.append(df2['competitors'][0][0]['id'])
      rows.append(df2['competitors'][0][1]['id'])
      try:
        if df2['competitors'][0][0]['winner'] == True:
          rows.append('Home')
        else:
          rows.append('Away')
      except KeyError:
        rows.append('NaN')
    except KeyError:
      print('it doesnt worked')
    Data_basic.loc[len(Data_basic)] = rows

     


In [18]:
for x in seasons:  
  for i in week:
    infos(x,2,i)

In [19]:
Data_basic

,GameID,season,types,week,HomeTeam_ID,AwayTeam_ID,Winner
0,400791485,2015,2,1,17,23,Home
1,400791541,2015,2,1,2,11,Home
2,400791489,2015,2,1,3,9,Away
3,400791550,2015,2,1,14,26,Home
4,400791497,2015,2,1,20,5,Home
...,...,...,...,...,...,...,...
2076,401437957,2022,2,18,21,19,Home
2077,401437962,2022,2,18,25,22,Home
2078,401437963,2022,2,18,26,14,Home
2079,401437959,2022,2,18,28,6,Home


In [40]:
playid_df = pd.DataFrame()
playid_df = pd.DataFrame(columns=['gameID','playID'])

In [41]:
def playid(GameID):
    url = f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
    response_drive = r.get(url)
    drive_data = response_drive.json()
    drive_df = json_normalize(drive_data['items'])
    for index, row in drive_df.iterrows():
      try:
        for play in range(len(row['plays.items'])):
          data = []
          playid = row['plays.items'][play]['id']
          data = [GameID, playid]
          playid_df.loc[len(playid_df)] = data
      except TypeError:
          print(GameID)

In [42]:
for games in Data_basic['GameID']:
  playid(games)

KeyError: ignored

In [43]:
playid_df

,gameID,playID
0,400791485,40079148536
1,400791485,40079148551
2,400791485,40079148572
3,400791485,400791485101
4,400791485,400791485122
...,...,...
284571,401326460,4013264603708
284572,401326460,4013264603729
284573,401326460,4013264603760
284574,401326460,4013264603779


In [45]:
playid_df.to_csv('/content/drive/MyDrive/gameid_playid_data.csv', index=False)